In [1]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
#import seaborn as sns
#from surprise import Reader, Dataset, SVD, evaluate
#sns.set_style("darkgrid")

In [2]:
import seaborn as sns

In [3]:
from surprise import Reader, Dataset, SVD, evaluate
sns.set_style("darkgrid")

In [4]:
df1 = pd.read_csv('combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'])
df2 = pd.read_csv('combined_data_2.txt', header = None, names = ['Cust_Id', 'Rating'])
df3 = pd.read_csv('combined_data_3.txt', header = None, names = ['Cust_Id', 'Rating'])
df4 = pd.read_csv('combined_data_4.txt', header = None, names = ['Cust_Id', 'Rating'])

In [5]:
def add_movieID(df, movie_id):
    df_nan = pd.DataFrame(pd.isnull(df.Rating))
    df_nan = df_nan[df_nan['Rating'] == True]
    df_nan = df_nan.reset_index()

    movie_np = []

    for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
        # numpy approach
        temp = np.full((1,i-j-1), movie_id)
        movie_np = np.append(movie_np, temp)
        movie_id += 1
        print(movie_id)

    # Account for last record and corresponding length
    # numpy approach
    last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
    movie_np = np.append(movie_np, last_record)

    print('Movie numpy: {}'.format(movie_np))
    print('Length: {}'.format(len(movie_np)))
    df = df[pd.notnull(df['Rating'])]

    df['Movie_Id'] = movie_np.astype(int)
    df['Cust_Id'] = df['Cust_Id'].astype(int)
    return df, movie_id

In [6]:
data = pd.DataFrame([])

In [4]:
tmp, movie_id = add_movieID(df1,1)
data = data.append(tmp)
tmp, movie_id = add_movieID(df2,movie_id)
data = data.append(tmp)
tmp, movie_id = add_movieID(df3,movie_id)
data = data.append(tmp)
tmp, movie_id = add_movieID(df4,movie_id)
data = data.append(tmp)

In [8]:
df = data

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(df[['Cust_Id', 'Movie_Id', 'Rating']][:len(df)])

In [9]:
probe = pd.read_csv("probe.txt",  names = ['Cust_Id'])

In [10]:
probe_movie = pd.DataFrame([])

In [11]:
Cust_Id = []
movie_id = []
currentMovie =0
index = 0

In [3]:
elem = 1
for i in probe.Cust_Id:
    if i[len(i) -1] == ":":
        elem = int(i[0:len(i)-1])
        print(elem)
        continue
    Cust_Id.append(int(i))
    movie_id.append(elem)
    #print(index,row)

In [13]:
train = df

In [15]:
#print(train)

In [2]:
probe_movie['Cust_Id'] = Cust_Id
probe_movie['Movie_Id'] = movie_id


In [124]:
tp = probe_movie.sort_values(by = 'Movie_Id')

In [125]:
#test = tp[tp.Movie_Id <= 9209]

In [17]:
test = probe_movie

In [19]:
labels = pd.DataFrame(np.int64(train.Rating))

In [20]:
train = pd.concat([train.Cust_Id, train.Movie_Id], axis=1)

In [22]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LinearRegression

In [23]:
lgr = LinearRegression()

In [24]:
model = lgr.fit(train, labels)

In [26]:
pred = model.predict(test)

In [27]:
tmp = df

In [29]:
#print(pd.merge(tmp, test, on=['Cust_Id', 'Movie_Id']))

In [ ]:
ratings = pd.merge(df,test, on=['Cust_Id','Movie_Id'])

In [166]:
test = pd.merge(tmp,test, on=['Cust_Id','Movie_Id'])

In [168]:
test.drop(['Rating'], axis = 1, inplace = True)

In [31]:
pred = model.predict(test)

In [32]:
ratings = pd.DataFrame(np.int64(ratings.Rating))

In [35]:
pred = pd.DataFrame(pred)

In [33]:
print(pred)

[[ 3.58835606]
 [ 3.58755736]
 [ 3.58797369]
 ..., 
 [ 3.60559143]
 [ 3.60602764]
 [ 3.6059176 ]]


In [36]:
error_calc = pd.concat([ratings,pred], axis=1)

In [37]:
print(ratings.shape, pred.shape)

(585820, 1) (1408395, 1)


In [175]:
from math import sqrt

In [1]:
print(len(error_calc))

NameError: name 'error_calc' is not defined

In [177]:
count = 0
for i, j in error_calc.iterrows():
    count += pow(j.values[0] -j.values[1],2)
rmse = sqrt(count/len(error_calc))

In [178]:
print(rmse)

1.130103297749197
